# Setup the dataframe

In [1]:
import pandas as pd
import os
from datetime import datetime, timezone
import time
import matplotlib.pyplot as plt
import numpy as np

all_data = pd.DataFrame()

# Be careful with this. If there is more than one CSV file only the last one will be used. 
for file in os.listdir('./'):
    if file.endswith('csv'):
        all_data = pd.read_csv('./' + file, sep=';')
        print("Using file: " + file)


c:\users\benjj\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (643,644,645,646,647,648,649) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Using file: 2020_09_16_14_02_20.csv


## Setup the Date_Time_(GMT) column

In [2]:
start = time.time()

all_data['PLC_TIME(Timedate48)'] = all_data['PLC_TIME(Timedate48)'].astype(str)

def getUnixTime(date):
    return date.split(',')[0]

def convertToUTC(date):
    return datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')

def convertToGMT(unix_timestamp):
    utcTime = datetime.fromtimestamp(unix_timestamp, timezone.utc)
    localTime = utcTime.astimezone()
    return localTime.strftime("%Y-%m-%d %H:%M:%S")
    
all_data['Date_Time_(GMT)'] = all_data['PLC_TIME(Timedate48)'].apply(lambda x: getUnixTime(x))
all_data['Date_Time_(GMT)'] = all_data['Date_Time_(GMT)'].astype('int32')
all_data['Date_Time_(GMT)'] = all_data['Date_Time_(GMT)'].apply(lambda x: convertToGMT(x))
column_name = 'Date_Time_(GMT)'
first_column = all_data.pop(column_name)
all_data.insert(0, column_name, first_column)
all_data['Date_Time_(GMT)'] = pd.to_datetime(all_data['Date_Time_(GMT)'])
all_data = all_data.sort_values(by='Date_Time_(GMT)')

duration = time.time() - start
print("Total time: %10.2f" % duration)

all_data.tail()

Total time:       1.63


,Date_Time_(GMT),TIME,PLC_TIME_OFFSET(Integer8),PLC_TIME_CV(Enum2),PLC_TIME(Timedate48),EVR_VehicleNumber(Unsigned16),EVR_TIME_CV(Enum2),EVR_TIME(Timedate48),EVR_Speed(Unsigned16),EVR_Speed_CV(Enum2),...,EKE_DIOs_Car15(Bitset32),EKE_DIOs_Car16(Bitset32),EKE_DIOs_Car2(Bitset32),EKE_DIOs_Car3(Bitset32),EKE_DIOs_Car4(Bitset32),EKE_DIOs_Car5(Bitset32),EKE_DIOs_Car6(Bitset32),EKE_DIOs_Car7(Bitset32),EKE_DIOs_Car8(Bitset32),EKE_DIOs_Car9(Bitset32)
32496,2020-09-16 18:39:38,"16637,952",0,1,"1600277978,000",15002,1,"1600277978,000",0,1,...,0,0,31c083,31c083,31c083,31c083,31c083,31c083,431c083,0
32497,2020-09-16 18:39:39,"16638,464",0,1,"1600277979,000",15002,1,"1600277979,000",0,1,...,0,0,31c083,31c083,31c083,31c083,31c083,31c083,431c083,0
32498,2020-09-16 18:39:39,"16638,976",0,1,"1600277979,000",15002,1,"1600277979,000",0,1,...,0,0,31c083,31c083,31c083,31c083,31c083,31c083,431c083,0
32499,2020-09-16 18:39:40,"16639,488",0,1,"1600277980,000",15002,1,"1600277980,000",0,1,...,0,0,31c083,31c083,31c083,31c083,31c083,31c083,431c083,0
32500,2020-09-16 18:39:40,"16640,000",0,1,"1600277980,000",15002,1,"1600277980,000",0,1,...,0,0,31c083,31c083,31c083,31c083,31c083,31c083,431c083,0


#### Cleanup the NaN entries

In [3]:
print("Rows before: " + str(all_data.shape[0]))
all_data = all_data.dropna(how='all')
print("Rows after: " + str(all_data.shape[0]))
all_data.head()

Rows before: 32501
Rows after: 32501


,Date_Time_(GMT),TIME,PLC_TIME_OFFSET(Integer8),PLC_TIME_CV(Enum2),PLC_TIME(Timedate48),EVR_VehicleNumber(Unsigned16),EVR_TIME_CV(Enum2),EVR_TIME(Timedate48),EVR_Speed(Unsigned16),EVR_Speed_CV(Enum2),...,EKE_DIOs_Car15(Bitset32),EKE_DIOs_Car16(Bitset32),EKE_DIOs_Car2(Bitset32),EKE_DIOs_Car3(Bitset32),EKE_DIOs_Car4(Bitset32),EKE_DIOs_Car5(Bitset32),EKE_DIOs_Car6(Bitset32),EKE_DIOs_Car7(Bitset32),EKE_DIOs_Car8(Bitset32),EKE_DIOs_Car9(Bitset32)
0,2020-09-16 14:02:20,"0,000",0,1,"1600261340,000",15002,1,"1600261340,000",0,1,...,0,0,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,0
1,2020-09-16 14:02:20,"0,512",0,1,"1600261340,000",15002,1,"1600261341,000",0,1,...,0,0,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,0
2,2020-09-16 14:02:21,"1,024",0,1,"1600261341,000",15002,1,"1600261341,000",0,1,...,0,0,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,0
3,2020-09-16 14:02:21,"1,536",0,1,"1600261341,000",15002,1,"1600261342,000",0,1,...,0,0,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,0
4,2020-09-16 14:02:22,"2,048",0,1,"1600261342,000",15002,1,"1600261342,000",0,1,...,0,0,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,3f1c083,0


### Remove string columns

In [4]:
import numpy as np
HABD_cols = [col for col in all_data if not 'Date_Time_(GMT)' in col]
before = all_data.shape[1]

for col in HABD_cols:
    if all_data.dtypes[col] == np.object:
        del all_data[col]

print("Rows removed: " + str(before - all_data.shape[1]))
print("cols in all_data: " + str(all_data.shape[1]))

Rows removed: 11
cols in all_data: 641


### Remove 0 data columns

In [5]:
HABD_cols = [col for col in all_data if not 'Date_Time_(GMT)' in col]
before = all_data.shape[1]

for col in HABD_cols:
    if all_data[col].sum() == 0:
        del all_data[col]

print("Rows removed: " + str(before - all_data.shape[1]))
print("cols in all_data: " + str(all_data.shape[1]))

Rows removed: 451
cols in all_data: 190


### Setup HABD_columns for removal

In [7]:
HABD_columns = []
HABD_columns.append('EVR_VehicleNumber(Unsigned16)')   # Remove
HABD_columns.append('EVR_TIME_CV(Enum2)')
HABD_columns.append('EVR_Speed_CV(Enum2)')
HABD_columns.append('EVR_Distance(Unsigned32)')
HABD_columns.append('HABD_HighTemp(Unsigned16)')

print("cols in list: " + str(len(HABD_columns)))
print("cols in all_data: " + str(all_data.shape[1]))
print("rows in all_data: " + str(all_data.shape[0]))

cols in list: 5
cols in all_data: 190
rows in all_data: 32501


##### Remove HABD_columns

In [8]:
before = all_data.shape[1]
#all_data.drop(HABD_columns, axis=1)
for column in HABD_columns:
    del all_data[column]
# HABD_columns.pop(0) # removes useless column 115, 115 ...
print("Rows removed: " + str(before - all_data.shape[1]))
print("cols in all_data: " + str(all_data.shape[1]))
print("rows in all_data: " + str(all_data.shape[0]))

Rows removed: 5
cols in all_data: 185
rows in all_data: 32501


### Check the data counts

In [13]:
HABD_cols = [col for col in all_data if not 'NUMBER' in col]

for col in HABD_cols[1:]:
    print(all_data[col].value_counts())

1    32501
Name: PLC_TIME_CV(Enum2), dtype: int64
18    22194
19    10307
Name: HABD_TempCh10_Car1(Integer16), dtype: int64
19    29794
0      2618
20       89
Name: HABD_TempCh9_Car8(Integer16), dtype: int64
20    29881
0      2620
Name: HABD_TempCh9_Car7(Integer16), dtype: int64
19    24384
20     5497
0      2620
Name: HABD_TempCh9_Car6(Integer16), dtype: int64
18    24097
19     5784
0      2620
Name: HABD_TempCh9_Car5(Integer16), dtype: int64
20    29881
0      2620
Name: HABD_TempCh9_Car4(Integer16), dtype: int64
19    24316
20     3894
0      2618
18     1673
Name: HABD_TempCh9_Car3(Integer16), dtype: int64
18    23324
19     6557
0      2620
Name: HABD_TempCh9_Car2(Integer16), dtype: int64
19    32501
Name: HABD_TempCh9_Car1(Integer16), dtype: int64
20    15093
21    14790
0      2618
Name: HABD_TempCh8_Car8(Integer16), dtype: int64
19    29792
0      2620
20       89
Name: HABD_TempCh8_Car7(Integer16), dtype: int64
20    29881
0      2620
Name: HABD_TempCh8_Car6(Integer16), dt

### Check the temperature counts

In [14]:
HABD_cols = [col for col in all_data if 'Temp' in col]

for col in HABD_cols[1:]:
    print(all_data[col].value_counts())

19    29794
0      2618
20       89
Name: HABD_TempCh9_Car8(Integer16), dtype: int64
20    29881
0      2620
Name: HABD_TempCh9_Car7(Integer16), dtype: int64
19    24384
20     5497
0      2620
Name: HABD_TempCh9_Car6(Integer16), dtype: int64
18    24097
19     5784
0      2620
Name: HABD_TempCh9_Car5(Integer16), dtype: int64
20    29881
0      2620
Name: HABD_TempCh9_Car4(Integer16), dtype: int64
19    24316
20     3894
0      2618
18     1673
Name: HABD_TempCh9_Car3(Integer16), dtype: int64
18    23324
19     6557
0      2620
Name: HABD_TempCh9_Car2(Integer16), dtype: int64
19    32501
Name: HABD_TempCh9_Car1(Integer16), dtype: int64
20    15093
21    14790
0      2618
Name: HABD_TempCh8_Car8(Integer16), dtype: int64
19    29792
0      2620
20       89
Name: HABD_TempCh8_Car7(Integer16), dtype: int64
20    29881
0      2620
Name: HABD_TempCh8_Car6(Integer16), dtype: int64
18    18469
17    11412
0      2620
Name: HABD_TempCh8_Car5(Integer16), dtype: int64
19    29881
0      2620
Name

# Graph a coach's temperature spikes

#### Create a date mask for the time frame

In [31]:
start_date = '2020-09-16 14:02:20'
end_date = '2020-09-16 14:03:10'
mask = (all_data['Date_Time_(GMT)'] > start_date) & (all_data['Date_Time_(GMT)'] <= end_date)
print(all_data.loc[mask]) # select the masked timeframe

       Date_Time_(GMT)  PLC_TIME_CV(Enum2)  HABD_TempCh10_Car1(Integer16)  \
2  2020-09-16 14:02:21                   1                             19   
3  2020-09-16 14:02:21                   1                             19   
4  2020-09-16 14:02:22                   1                             19   
5  2020-09-16 14:02:22                   1                             19   
6  2020-09-16 14:02:23                   1                             19   
..                 ...                 ...                            ...   
95 2020-09-16 14:03:09                   1                             19   
96 2020-09-16 14:03:09                   1                             19   
97 2020-09-16 14:03:09                   1                             19   
98 2020-09-16 14:03:10                   1                             19   
99 2020-09-16 14:03:10                   1                             19   

    HABD_TempCh9_Car8(Integer16)  HABD_TempCh9_Car7(Integer16)  \
2        

In [24]:
all_data.head(100)

,Date_Time_(GMT),PLC_TIME_CV(Enum2),HABD_TempCh10_Car1(Integer16),HABD_TempCh9_Car8(Integer16),HABD_TempCh9_Car7(Integer16),HABD_TempCh9_Car6(Integer16),HABD_TempCh9_Car5(Integer16),HABD_TempCh9_Car4(Integer16),HABD_TempCh9_Car3(Integer16),HABD_TempCh9_Car2(Integer16),...,EKE_DIOs1_P[22](Unsigned8),EKE_DIOs1_P[23](Unsigned8),EKE_DIOs1_P[24](Unsigned8),EKE_DIOs1_P[25](Unsigned8),EKE_DIOs1_P[26](Unsigned8),EKE_DIOs1_P[27](Unsigned8),EKE_DIOs1_P[28](Unsigned8),EKE_DIOs1_P[29](Unsigned8),EKE_DIOs1_P[30](Unsigned8),EKE_DIOs1_P[31](Unsigned8)
0,2020-09-16 14:02:20,1,19,20,20,20,18,20,18,19,...,192,131,3,241,192,131,3,241,192,131
1,2020-09-16 14:02:20,1,19,20,20,20,18,20,18,19,...,192,131,3,241,192,131,3,241,192,131
2,2020-09-16 14:02:21,1,19,20,20,20,18,20,18,19,...,192,131,3,241,192,131,3,241,192,131
3,2020-09-16 14:02:21,1,19,20,20,20,18,20,18,19,...,192,131,3,241,192,131,3,241,192,131
4,2020-09-16 14:02:22,1,19,20,20,20,18,20,18,19,...,192,131,3,241,192,131,3,241,192,131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2020-09-16 14:03:09,1,19,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,2020-09-16 14:03:09,1,19,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,2020-09-16 14:03:09,1,19,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,2020-09-16 14:03:10,1,19,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:


df = pd.DataFrame(all_data, columns=[all_data['Date_Time_(GMT)'].loc[mask], all_data['HABD_TempCh9_Car8(Integer16)'].loc[mask]])
df.plot(x=all_data['Date_Time_(GMT)'].loc[mask], y=all_data['HABD_TempCh9_Car8(Integer16)'].loc[mask], kind='line' )
plt.show()

NotImplementedError: Index._join_level on non-unique index is not implemented

In [18]:
all_data.head(2)

,Date_Time_(GMT),PLC_TIME_CV(Enum2),HABD_TempCh10_Car1(Integer16),HABD_TempCh9_Car8(Integer16),HABD_TempCh9_Car7(Integer16),HABD_TempCh9_Car6(Integer16),HABD_TempCh9_Car5(Integer16),HABD_TempCh9_Car4(Integer16),HABD_TempCh9_Car3(Integer16),HABD_TempCh9_Car2(Integer16),...,EKE_DIOs1_P[22](Unsigned8),EKE_DIOs1_P[23](Unsigned8),EKE_DIOs1_P[24](Unsigned8),EKE_DIOs1_P[25](Unsigned8),EKE_DIOs1_P[26](Unsigned8),EKE_DIOs1_P[27](Unsigned8),EKE_DIOs1_P[28](Unsigned8),EKE_DIOs1_P[29](Unsigned8),EKE_DIOs1_P[30](Unsigned8),EKE_DIOs1_P[31](Unsigned8)
0,2020-09-16 14:02:20,1,19,20,20,20,18,20,18,19,...,192,131,3,241,192,131,3,241,192,131
1,2020-09-16 14:02:20,1,19,20,20,20,18,20,18,19,...,192,131,3,241,192,131,3,241,192,131


### Filter out the low temperatures

In [11]:
HABD_cols = [col for col in all_data]


In [12]:
print(HABD_cols)

['Date_Time_(GMT)', 'PLC_TIME_CV(Enum2)', 'HABD_TempCh10_Car1(Integer16)', 'HABD_TempCh9_Car8(Integer16)', 'HABD_TempCh9_Car7(Integer16)', 'HABD_TempCh9_Car6(Integer16)', 'HABD_TempCh9_Car5(Integer16)', 'HABD_TempCh9_Car4(Integer16)', 'HABD_TempCh9_Car3(Integer16)', 'HABD_TempCh9_Car2(Integer16)', 'HABD_TempCh9_Car1(Integer16)', 'HABD_TempCh8_Car8(Integer16)', 'HABD_TempCh8_Car7(Integer16)', 'HABD_TempCh8_Car6(Integer16)', 'HABD_TempCh8_Car5(Integer16)', 'HABD_TempCh8_Car4(Integer16)', 'HABD_TempCh8_Car3(Integer16)', 'HABD_TempCh8_Car2(Integer16)', 'HABD_TempCh8_Car1(Integer16)', 'HABD_TempCh7_Car8(Integer16)', 'HABD_TempCh7_Car7(Integer16)', 'HABD_TempCh7_Car6(Integer16)', 'HABD_TempCh7_Car5(Integer16)', 'HABD_TempCh7_Car4(Integer16)', 'HABD_TempCh7_Car3(Integer16)', 'HABD_TempCh7_Car2(Integer16)', 'HABD_TempCh7_Car1(Integer16)', 'HABD_TempCh6_Car8(Integer16)', 'HABD_TempCh6_Car7(Integer16)', 'HABD_TempCh6_Car6(Integer16)', 'HABD_TempCh6_Car5(Integer16)', 'HABD_TempCh6_Car4(Integer16)